In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import os
import math
import data_loader
import ResNet as models
from torch.utils.tensorboard import SummaryWriter
from config.Domain_Adapt_settings import *
import pandas as pd
import dataset_loader as dl

In [2]:
torch.cuda.empty_cache()
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# torch.cuda.set_device(1)
writer = SummaryWriter('runs')

# Training settings
st = Settings()
options = st.get_options()

batch_size = options['batch_size']
epochs = options['max_epochs']
lr = [0.001, 0.01]
momentum = 0.9
no_cuda =False
seed = options['seed']
log_interval = 10
l2_decay = 5e-4
source_train_path = options['source_train_folder']
target_train_path = options['target_train_folder']
source_name = 'miccai'
target_name = "ISBI"
cuda = not no_cuda and torch.cuda.is_available()

_, source_train_list, _ = next(os.walk(source_train_path))
_, target_train_list, _ = next(os.walk(target_train_path))

source_train_list.sort()
target_train_list.sort()

In [3]:
source_train_x_data = {f: {m: os.path.join(options['source_train_folder'], f, options['tmp_folder'], n)
                    for m, n in zip(options['modalities'], options['preprocess_x_names'])}
                for f in source_train_list}
source_train_y_data = {f: os.path.join(options['source_train_folder'], f, options['tmp_folder'],
                                options['preprocess_y_names'][0])
                for f in source_train_list}

target_train_x_data = {f: {m: os.path.join(options['target_train_folder'], f, options['tmp_folder'], n)
                    for m, n in zip(options['modalities'], options['preprocess_x_names'])}
                for f in target_train_list}

In [4]:
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

In [7]:
source_train_loader = dl.load_training(options, source_train_x_data, source_train_y_data)

> CNN: loading training data for first model


/home/mostafa/Marwa/Code/deep-transfer-learning/UDA/pytorch0.3/DAN/utils/data_load.py:362: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  patches = [new_image[idx] for idx in slices]


shape (1093132, 16, 16, 16)
> CNN: train_x  (1093132, 2, 16, 16, 16) train_y  (1093132,)


In [ ]:
target_train_loader = dl.load_training(options, target_train_x_data, y_data=None)

> CNN: loading training data for first model


In [ ]:
len_source_train_dataset = len(source_train_loader.dataset)
len_source_train_loader = len(source_train_loader)
print(len_source_train_dataset)
print(len_source_train_loader)

len_target_train_dataset = len(target_train_loader.dataset)
len_target_train_loader = len(target_train_loader)
print(len_target_train_dataset)
print(len_target_train_loader)

In [5]:
model = models.DANNet(num_classes=2)

In [6]:
FILE = os.path.join(options['weight_paths'], options['source_experiment'], options['pre_trained_model'])
print(FILE)
model = torch.load(FILE)

/home/mostafa/Marwa/Code/deep-transfer-learning/UDA/pytorch0.3/DAN/weights/resnet_DAN_full_miccai_train/2_model.pth


In [13]:
_, source_train_list, _ = next(os.walk(source_train_path))
_, target_train_list, _ = next(os.walk(target_train_path))

source_train_list.sort()
target_train_list.sort()

print(source_train_list)
print(target_train_list)

['CHB_train_Case01', 'CHB_train_Case02', 'CHB_train_Case03', 'CHB_train_Case04', 'CHB_train_Case05', 'CHB_train_Case06', 'CHB_train_Case07', 'CHB_train_Case08', 'CHB_train_Case09', 'CHB_train_Case10', 'Center_01_Patient_01', 'Center_01_Patient_02', 'Center_01_Patient_03', 'Center_01_Patient_04', 'Center_01_Patient_05', 'Center_07_Patient_01', 'Center_07_Patient_02', 'Center_07_Patient_03', 'Center_07_Patient_04', 'Center_07_Patient_05', 'Center_08_Patient_01', 'Center_08_Patient_02', 'Center_08_Patient_03', 'Center_08_Patient_04', 'Center_08_Patient_05', 'UNC_train_Case01', 'UNC_train_Case02', 'UNC_train_Case03', 'UNC_train_Case04', 'UNC_train_Case05', 'UNC_train_Case06', 'UNC_train_Case07', 'UNC_train_Case08', 'UNC_train_Case09', 'UNC_train_Case10']
['training01_01', 'training01_02', 'training01_03', 'training01_04', 'training02_01', 'training02_02', 'training02_03', 'training02_04', 'training03_01', 'training03_02', 'training03_03', 'training03_04', 'training03_05', 'training04_01', 

In [14]:
source_train_x_data = {f: {m: os.path.join(options['source_train_folder'], f, options['tmp_folder'], n)
                    for m, n in zip(options['modalities'], options['preprocess_x_names'])}
                for f in source_train_list}
source_train_y_data = {f: os.path.join(options['source_train_folder'], f, options['tmp_folder'],
                                options['preprocess_y_names'][0])
                for f in source_train_list}

target_train_x_data = {f: {m: os.path.join(options['target_train_folder'], f, options['tmp_folder'], n)
                    for m, n in zip(options['modalities'], options['preprocess_x_names'])}
                for f in target_train_list}

In [17]:
target_train_x_data

{'training01_01': {'FLAIR': '/home/mostafa/Marwa/DataSets/ISBI/train/training01_01/tmp/FLAIR_brain.nii.gz',
  'T1': '/home/mostafa/Marwa/DataSets/ISBI/train/training01_01/tmp/T1_brain.nii.gz'},
 'training01_02': {'FLAIR': '/home/mostafa/Marwa/DataSets/ISBI/train/training01_02/tmp/FLAIR_brain.nii.gz',
  'T1': '/home/mostafa/Marwa/DataSets/ISBI/train/training01_02/tmp/T1_brain.nii.gz'},
 'training01_03': {'FLAIR': '/home/mostafa/Marwa/DataSets/ISBI/train/training01_03/tmp/FLAIR_brain.nii.gz',
  'T1': '/home/mostafa/Marwa/DataSets/ISBI/train/training01_03/tmp/T1_brain.nii.gz'},
 'training01_04': {'FLAIR': '/home/mostafa/Marwa/DataSets/ISBI/train/training01_04/tmp/FLAIR_brain.nii.gz',
  'T1': '/home/mostafa/Marwa/DataSets/ISBI/train/training01_04/tmp/T1_brain.nii.gz'},
 'training02_01': {'FLAIR': '/home/mostafa/Marwa/DataSets/ISBI/train/training02_01/tmp/FLAIR_brain.nii.gz',
  'T1': '/home/mostafa/Marwa/DataSets/ISBI/train/training02_01/tmp/T1_brain.nii.gz'},
 'training02_02': {'FLAIR': '/

In [26]:
from nibabel import load as load_nii
from utils.data_load import *
train_x_data = target_train_x_data
# get_scan names and number of modalities used
scans = list(target_train_x_data.keys())
modalities = train_x_data[scans[0]].keys()
flair_scans = [train_x_data[s]['FLAIR'] for s in scans]
# load images and normalize their intensities
images = [load_nii(image_name).get_data() for image_name in flair_scans]
images_norm = [normalize_data(im) for im in images]

/home/mostafa/Marwa/Environments/deep_transfer_learning_env/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  if __name__ == '__main__':


In [29]:
# select voxels with intensity higher than threshold
selected_voxels = [image > options['min_th'] for image in images_norm]

In [34]:
print(len(selected_voxels), selected_voxels[0].shape)
data = []

for m in modalities:
    x_data = [train_x_data[s][m] for s in scans]
random_state=42
datatype=np.float32
patch_size = options['patch_size']
# Get all the x,y,z coordinates for each image
centers = [get_mask_voxels(mask) for mask in selected_voxels]

patches = [np.array(get_patches(image, centers, patch_size))
                 for image, centers in zip(images_norm, centers)]

21 (181, 217, 181)


/home/mostafa/Marwa/Code/deep-transfer-learning/UDA/pytorch0.3/DAN/utils/data_load.py:362: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  patches = [new_image[idx] for idx in slices]


In [38]:
print(len(centers[0]))
print(181 * 217* 181)

256798
7109137


In [ ]:


    
    data = []

    for m in modalities:
        x_data = [train_x_data[s][m] for s in scans]
    